# SimPy Example for IE 306.02
This example shows how a simple G/G/1 system (1 server with an infinite capacity queue, random interarrival and service times) can be modeled with a process-interaction view using the SimPy library. 

In this example customers place calls to the call center of a company at randomn times. There is only a single operator in this call center, and picks up the first call waiting when she is available. The customers are assumed to be extremely paint, as they wait as long as it gets to talk to the operator.

In [81]:
!pip install simpy

In [82]:
import simpy
import random
import math

Define a set of globals that define the characteristics of the model instance to be simulated. This includes the seed (RANDOM_SEED) for the random number generators, and key parameters for the interarrival (i.e. mean arrival rate) and service time (i.e. lower and upper bounds for the range) distribution.

In [83]:
Total_Time = 100000

RANDOM_SEED = 978
INTERARRIVAL_RATE = 0.1 
random.seed(RANDOM_SEED)

hospital_treating_time_rate = 6
home_treating_time_rate = 10
hospital2home_treating_time_rate = 6*r

num_of_individuals = 0
S = 147 + 177 + 207
if(S > 1000): num_of_individuals = S
elif(S > 10): num_of_individuals = S + 1000
else: num_of_individuals = S*300

num_of_beds = math.ceil(num_of_individuals/12)
exponential_rate = num_of_individuals/300

num_of_patients = math.ceil(random.expovariate(num_of_individuals/Total_Time))
print(num_of_individuals, num_of_patients, r)

1531 289 1.9879529669203548


Define the necessary set of arrays for bookkeeping

In [84]:
service_times = [] #Duration of the conversation between the customer and the operator (Service time)

* The class definition for the customers arriving at the modeled system. When they are created, they immediatelly initiate a call (i.e. activate the call process). 

* Once a call is initiated, this is registered as a request to the operator resource. The customer is put on hold until the resource activates it back. 

* When the resource is available, the customer is activated and it then initiates the ask_question process. The duration of a question-answer session is determined randomly according to a uniform distribution.

In [85]:
class Patient(object):
    def __init__(self, name, env, opr):
        self.env = env
        self.name = name
        self.arrival_t = self.env.now
        self.r = random.uniform(1,2)
        self.action = env.process(self.try_to_heal())
    
    
    def try_to_heal(self):
        if(self.is_going_hospital() == False):
            print('%s at home at %g' % (self.name, math.ceil(self.env.now)))
            yield self.env.process(self.healing())
            print('%s is healed at %g in home' % (self.name, math.ceil(self.env.now)))
        else:
            print('%s arrives hospital at %g' % (self.name, math.ceil(self.env.now)))
            with bed.request() as req:
                yield req
                print('%s is assigned to an bed at %g' % (self.name, math.ceil(self.env.now)))
                yield self.env.process(self.healing())
                print('%s is healed at %g' % (self.name, math.ceil(self.env.now)))
            
            
    def healing(self):
        if(self.is_going_hospital and bed.count != num_of_beds):
            duration = random.expovariate(1.0/hospital_treating_time_rate)
        elif(self.is_going_hospital):
            duration = random.expovariate(1.0/hospital2home_treating_time_rate)
        else:
            duration = random.expovariate(1.0/home_treating_time_rate)
        yield self.env.timeout(duration)
        service_times.append(duration)
        
        
    def is_going_hospital(self):
        random_number = random.random()
        if(random_number > 0.2): return False
        else: return True
    

In [86]:
def patient_generator(env, bed):
    """Generate new patients with using exponential rate!!!."""
    for i in range(num_of_patients):
        yield env.timeout(random.expovariate(INTERARRIVAL_RATE))
        patient = Patient('Patient %s' %(i+1), env, bed)  

In [ ]:
env = simpy.Environment()
bed = simpy.Resource(env, capacity = num_of_beds)
env.process(patient_generator(env, bed))
env.run()

In [89]:
print(service_times)

[3.432674235011879, 3.833283246874041, 14.12352231597659, 7.829072395722217, 11.678090290993989, 0.22589314964950116, 0.12786088103587684, 15.77657276015972, 5.198171690638639, 10.549840189330729, 2.5700611525207857, 2.131818946649559, 7.1729546500174575, 14.989611130023462, 4.007450516112128, 3.988555798949945, 4.385147906115849, 7.776439502486757, 6.632814722752768, 2.235684504711231, 2.5357802111726504, 11.646375142703876, 2.233128072592993, 6.7410500822070585, 0.5395018342564907, 1.4527597235052887, 2.8477896730697156, 26.277517852551966, 0.3398495874489467, 3.2375149750463854, 2.877034275308843, 21.06344273258537, 0.2729295670532327, 20.68357092172196, 12.569266349786746, 7.9628435758004334, 0.5816818582693353, 5.392100977754625, 3.55324863194173, 4.434231646136567, 17.679874093430726, 2.4857261556314496, 3.9237297482015125, 7.728609868564054, 2.6416273671216697, 5.927719535417098, 3.534479552549958, 16.423838242917405, 4.496378179436732, 13.879022072976266, 7.244217798514993, 0.1